In [24]:
import tensorflow as tf
from tensorflow.keras.layers import average, maximum
import tensorflow_hub as hub
import os
import numpy as np
import tensorflow_hub as hub
import pathlib

def read_and_label(file_path):
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    label = get_label(file_path)
    return img, label


def decode_img(img):
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])


def get_label(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    return tf.reshape(tf.where(parts[-4] == CLASS_NAMES), [])

def augment(image, label):
    image = tf.image.random_hue(image, max_delta=0.05, seed=5)
    image = tf.image.random_contrast(image, 0.95, 1.05, seed=5)  # tissue quality
    image = tf.image.random_saturation(image, 0.95, 1.05, seed=5)  # stain quality
    image = tf.image.random_brightness(image, max_delta=0.05)  # tissue thickness, glass transparency (clean)
    image = tf.image.random_flip_left_right(image, seed=5)  # cell orientation
    image = tf.image.random_flip_up_down(image, seed=5)  # cell orientation
    image = tf.image.rot90(image, tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))  # cell orientation
    return image, label


IMG_HEIGHT = 100
IMG_WIDTH = 100
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 64
shuffle_buffer_size = 1000000

test_data_dir = r'C:\Users\kuki\Desktop\Research\Skin\RCNN data\test'
test_data_dir = pathlib.Path(test_data_dir)
CLASS_NAMES = np.array(
    [item.name for item in test_data_dir.glob('*') if item.name != "LICENSE.txt" and item.name != ".DS_store"])


lr = 'e4'
model_cnnA = tf.keras.models.load_model('cnn/Alex/'+lr+'/full_model.h5', compile=False)

model_cnnA.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])


def load_dataset(dataset_dir):
    dataset_dir = pathlib.Path(dataset_dir)
    test_image_count2 = len(list(test_data_dir.glob('image\*.jpg')))
    list_ds = tf.data.Dataset.list_files(str(dataset_dir / 'image/*.jpg'))
    # for f in list_ds.take(5):
    #     print(f.numpy())
    labeled_ds = list_ds.map(read_and_label, num_parallel_calls=AUTOTUNE)
    return labeled_ds, test_image_count2


def evalmodels(path):
    datasett, datasettsize = load_dataset(path)
    results = model_cnnA.evaluate(datasett.batch(1000))
    print(os.path.basename(path), results[-1] * 100)


# evalmodels(r'C:\Users\kuki\Desktop\Research\Skin\RCNN data\train\young\sec001')
# print('---------------------------------------------------------------------------')
# evalmodels(r'C:\Users\kuki\Desktop\Research\Skin\RCNN data\train\young\sec003')
# print('---------------------------------------------------------------------------')
# evalmodels(r'C:\Users\kuki\Desktop\Research\Skin\RCNN data\train\young\sec007')
# print('---------------------------------------------------------------------------')
# evalmodels(r'C:\Users\kuki\Desktop\Research\Skin\RCNN data\train\young\sec010')
# print('---------------------------------------------------------------------------')
# evalmodels(r'C:\Users\kuki\Desktop\Research\Skin\RCNN data\train\young\sec016')
# print('---------------------------------------------------------------------------')
# evalmodels(r'C:\Users\kuki\Desktop\Research\Skin\RCNN data\train\young\sec019')
# print('---------------------------------------------------------------------------')
#
#
#
# evalmodels(r'C:\Users\kuki\Desktop\Research\Skin\RCNN data\test\young\sec023')
# print('---------------------------------------------------------------------------')
# evalmodels(r'C:\Users\kuki\Desktop\Research\Skin\RCNN data\test\young\sec025')
# print('---------------------------------------------------------------------------')
# evalmodels(r'C:\Users\kuki\Desktop\Research\Skin\RCNN data\test\young\sec029')
# print('---------------------------------------------------------------------------')
#
#
# evalmodels(r'C:\Users\kuki\Desktop\Research\Skin\RCNN data\train\old\sec031')
# print('---------------------------------------------------------------------------')
# evalmodels(r'C:\Users\kuki\Desktop\Research\Skin\RCNN data\train\old\sec037')
# print('---------------------------------------------------------------------------')
# evalmodels(r'C:\Users\kuki\Desktop\Research\Skin\RCNN data\train\old\sec041')
# print('---------------------------------------------------------------------------')
# evalmodels(r'C:\Users\kuki\Desktop\Research\Skin\RCNN data\train\old\sec045')
# print('---------------------------------------------------------------------------')
# evalmodels(r'C:\Users\kuki\Desktop\Research\Skin\RCNN data\train\old\sec049')
# print('---------------------------------------------------------------------------')
# evalmodels(r'C:\Users\kuki\Desktop\Research\Skin\RCNN data\train\old\sec062')
# print('---------------------------------------------------------------------------')
# evalmodels(r'C:\Users\kuki\Desktop\Research\Skin\RCNN data\train\old\sec068')
# print('---------------------------------------------------------------------------')
# evalmodels(r'C:\Users\kuki\Desktop\Research\Skin\RCNN data\train\old\sec070')
# print('---------------------------------------------------------------------------')
#
# evalmodels(r'C:\Users\kuki\Desktop\Research\Skin\RCNN data\test\old\sec076')
# print('---------------------------------------------------------------------------')
# evalmodels(r'C:\Users\kuki\Desktop\Research\Skin\RCNN data\test\old\sec078')
# print('---------------------------------------------------------------------------')
# evalmodels(r'C:\Users\kuki\Desktop\Research\Skin\RCNN data\test\old\sec082')
# print('---------------------------------------------------------------------------')
# evalmodels(r'C:\Users\kuki\Desktop\Research\Skin\RCNN data\test\old\sec088')
# print('---------------------------------------------------------------------------')
#


def balance(data_dir):
    tmp = [0]
    for CLASS, n in zip(CLASS_NAMES, samplesize):
        secs = [_ for _ in data_dir.glob(CLASS+'/*')]
        for idx,sec in enumerate(secs):
            sec = os.path.join(sec,'image\*.jpg')
            list_ds = tf.data.Dataset.list_files(sec)
            # subsample
            list_ds = (list_ds
                       .shuffle(shuffle_buffer_size)
                       .take(n)
                       )
            labeled_ds = list_ds.map(read_and_label, num_parallel_calls=AUTOTUNE)

            # add augment
            sampleN = len(list(labeled_ds))
            while sampleN < n:
                labeled_ds_aug = (labeled_ds
                                  .shuffle(shuffle_buffer_size)
                                  .take(n-sampleN)
                                  .map(augment,num_parallel_calls=AUTOTUNE)
                                  )
                labeled_ds = labeled_ds.concatenate(labeled_ds_aug)
                sampleN = len(list(labeled_ds))
            print('list_ds: ',len(list(labeled_ds)),CLASS)
            # append
            if tmp[0]==0:
                tmp[idx]=labeled_ds
            else:
                labeled_ds = tmp[0].concatenate(labeled_ds)
                tmp[0]=labeled_ds
        print(CLASS,': sample size =',len(list(tmp[0])))
    return tmp[0].shuffle(shuffle_buffer_size)

test_data_dir = r'C:\Users\kuki\Desktop\Research\Skin\RCNN data\test'
test_data_dir = pathlib.Path(test_data_dir)
samplesize=[300,400]
test_labeled_ds = balance(test_data_dir)

test_ds = (test_labeled_ds
           .cache("./cache/fibro_test.tfcache")
           .shuffle(buffer_size=shuffle_buffer_size)
           .repeat()
           .batch(BATCH_SIZE)
           .prefetch(buffer_size=AUTOTUNE)  # time it takes to produce next element
           )
test_image_count = len(list(test_labeled_ds))
print('test set size : ', test_image_count)
TEST_STEPS = test_image_count // BATCH_SIZE

results = model_cnnA.evaluate(test_ds, steps=TEST_STEPS)

probability_model = tf.keras.Sequential([model_cnnA,
                                         tf.keras.layers.Softmax()])

list_ds:  300 old
list_ds:  300 old
list_ds:  300 old
list_ds:  300 old
old : sample size = 1200
list_ds:  400 young
list_ds:  400 young
list_ds:  400 young
young : sample size = 2400
test set size :  2400
37/37 [==============================] - 3s 68ms/step - loss: 52.1010 - accuracy: 0.5000


In [25]:
predictions = probability_model.predict(test_ds,steps=37)

In [26]:
for img,label in test_ds.take(100).as_numpy_iterator():
    print(label)

[0 0 0 0 0 1 0 0 1 1 1 1 0 0 1 1 1 1 0 0 1 0 0 1 0 1 0 1 0 0 1 1 1 1 1 1 0
 0 1 0 1 0 0 1 1 0 1 1 1 1 0 0 1 1 0 1 1 0 1 0 0 1 0 0]
[0 1 1 0 0 1 1 1 0 1 0 0 1 1 1 0 0 1 0 0 1 1 0 0 1 0 0 0 1 0 0 1 0 0 1 1 1
 1 1 0 0 1 1 1 0 0 1 0 0 1 1 1 1 1 0 1 1 0 0 1 1 1 0 1]
[1 0 1 1 1 1 0 0 1 0 0 0 0 0 0 0 1 0 1 1 0 0 0 1 0 0 0 0 0 1 0 1 0 1 0 0 1
 1 1 1 1 0 0 0 0 0 0 0 1 0 1 0 1 0 1 1 1 1 0 0 0 1 0 0]
[0 0 1 0 0 1 1 1 1 0 1 0 0 0 1 0 1 1 0 0 0 0 0 1 0 1 0 0 0 1 1 1 0 0 1 0 1
 0 1 1 0 1 1 1 0 1 1 0 0 0 1 1 0 1 1 1 0 1 0 1 0 0 1 0]
[1 0 1 1 0 1 0 0 1 0 1 0 1 1 1 1 0 0 0 1 0 1 1 0 0 0 1 1 1 0 1 1 1 1 0 0 1
 1 1 0 1 0 1 0 0 0 0 1 1 0 1 1 0 0 0 1 1 0 1 0 0 1 1 0]
[1 0 0 0 0 0 0 1 1 1 1 0 1 0 1 1 1 1 1 1 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 1
 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 1 0 0 0 1 1 1 0 0 0 0 0]
[0 0 1 1 0 1 0 1 1 1 1 1 1 0 1 0 0 0 1 0 0 1 1 1 1 1 1 1 0 1 1 1 0 0 0 0 0
 1 1 1 0 1 1 1 0 0 1 0 1 1 0 1 1 1 0 0 1 1 0 0 0 0 1 0]
[1 0 1 0 0 1 0 1 0 1 1 0 1 0 0 1 0 0 1 1 1 1 1 0 1 1 0 1 0 1 1 1 0 1 0 0 1
 1 1 1 1